# Pre-process input data for coastal variable extraction

Author: Emily Sturdivant; esturdivant@usgs.gov

***

Pre-process files to be used in extractor.ipynb (Extract barrier island metrics along transects). See the project [README](https://github.com/esturdivant-usgs/BI-geomorph-extraction/blob/master/README.md) and the Methods Report (Zeigler et al., in review). 


## Pre-processing steps

1. Pre-created geomorphic features: dunes, shoreline points, armoring.
2. Inlets
3. Shoreline
4. Transects - extend and sort
5. Transects - tidy


## Notes:
This process requires some manipulation of the spatial layers by the user. When applicable, instructions are described in this file.

***

## Import modules

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import arcpy
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import core.functions_warcpy as fwa
import core.functions as fun

In [2]:
from core.setvars import *

site (options: CapeLookout, Rockaway, Fisherman, Monomoy, ShipShoal, CapeHatteras, Parramore, FireIsland, Assateague, ParkerRiver, Cobb, Cedar, Metompkin, Smith, CoastGuard, RhodeIsland, Wreck, Myrtle, Forsythe, Assawoman):  Monomoy
year (options: 2010, 2012, 2014):  2014
Path to project directory (e.g. \\Macolume\dir\FireIsland2014):  ················································


setvars.py initialized variables.
SITE: Monomoy
MHW: 0.39
MLW: -0.95
Max dune crest height: 3
Projection code: 26919



### Initialize variables

Change the filename variables to match your local files. They should be in an Esri file geodatabase named site+year.gdb in your project directory, which will be the value of the variable `home`. 

Input the site, year, and project directory path. `setvars.py` retrieves the pre-determined values for that site in that year from `configmap.py`. The project directory will be used to set up your workspace. It's hidden for security – sorry! I recommend that you type the path somewhere and paste it in.

In [3]:
# Inputs - vector
orig_trans = os.path.join(home, 'trans_orig')

# Extended transects: NASC transects extended and sorted, ready to be the base geometry for processing
extendedTrans = os.path.join(home, 'Monomoy2014_extTrans_null')

# Tidied transects: Extended transects without overlapping transects
extTrans_tidy = os.path.join(home, 'Monomoy_tidyTrans') 

# Geomorphology points: positions of indicated geomorphic features
ShorelinePts = os.path.join(home, 'Monomoy2014_SLpts')  # shoreline
dlPts = os.path.join(home, 'Monomoy2014_DLpts')         # dune toe
dhPts = os.path.join(home, 'Monomoy2014_DHpts')         # dune crest

# Inlet lines: polyline feature classes delimiting inlet position. Must intersect the full island shoreline
inletLines = os.path.join(home, 'Monomoy2014_inletLines')

# Full island shoreline: polygon that outlines the island shoreline, MHW on oceanside and MTL on bayside
barrierBoundary = os.path.join(home, 'bndpoly_manual_2sl')  

# Elevation grid: DEM of island elevation at either 5 m or 1 m resolution
elevGrid = os.path.join(home, 'Monomoy2014_DEM_5m')


# ---
# OPTIONAL - comment out each one that is not available
# ---
# Study area boundary; manually digitize if the barrier island study area does not end at an inlet.
# SA_bounds = os.path.join(home, 'SA_bounds')

# Armoring lines: digitize lines of shorefront armoring to be used if dune toe points are not available.
# armorLines = os.path.join(home, 'armorLines')

## Prepare input layers

In [ ]:
datapre = '14CNT01'
csvpath = os.path.join(proj_dir, 'Input_Data', '{}_morphology'.format(datapre), '{}_morphology.csv'.format(datapre))

# state = sitevals['state']
dt_fc, dc_fc, sl_fc = fwa.MorphologyCSV_to_FCsByFeature(csvpath, state, proj_code, csv_fill = 999, fc_fill = -99999, csv_epsg=4326)

### Dunes
Display the points and the DEM in a GIS to check for irregularities. For example, if shoreline points representing a distance less than X m are visually offset from the general shoreline, they should likely be removed. Another red flag is when the positions of dlows and dhighs in relation to the shore are illogical, i.e. dune crests are seaward of dune toes. Address these irregularities by manually deleting points. Delete conservatively.

### Armoring
If the dlows do not capture the entire top-of-beach due to atypical formations caused by anthropogenic modification, you may need to digitize the beachfront armoring. The next code block will generate an empty feature class. Refer to the DEM and orthoimagery. If there is no armoring in the study area, continue. If there is armoring, use the Editing toolbar to add lines to the feature class that trace instances of armoring. Common manifestations of what we call armoring are sandfencing and sandbagging and concrete seawalls. 

If there is no armoring file in the project geodatabase, the extractor script will notify you that it is proceeding without armoring.

*__Requires manipulation in GIS__*

In [ ]:
arcpy.CreateFeatureclass_management(home, os.path.basename(armorLines), 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now manually digitize the shorefront armoring.".format(armorLines))

### Inlets
We also need to manually digitize inlets if an inlet delineation does not already exist. To do, the code below will produce the feature class. After which, use the Editing toolbar to create a line where the oceanside shore meets a tidal inlet. If the study area includes both sides of an inlet, that inlet will be represented by two lines. The inlet lines are used to define the bounds of the oceanside shore, which is also considered the point where the oceanside shore meets the bayside. Inlet lines must intersect the MHW contour. 

What do we do when the study area and not an inlet is the end?

*__Requires manipulation in GIS__*

In [ ]:
# manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
arcpy.CreateFeatureclass_management(home, os.path.basename(inletLines), 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now we'll stop for you to manually create lines at each inlet.".format(inletLines))

### Shoreline
The shoreline is produced through a combination of the DEM and the shoreline points. The first step converts the DEM to both MTL and MHW contour polygons. Those polygons are combined to produce the full shoreline, which is considered to fall at MHW on the oceanside and MTL on the bayside (to include partially submerged wetland).

If the study area does not end cleanly at an inlet, create a separate polyline feature class (default name is 'SA_bounds') and add lines that bisect the shoreline; they should look and function like inlet lines. Specify this in the arguments for DEMtoFullShorelinePoly() and CreateShoreBetweenInlets().

At some small inlets, channel depth may be above MTL. In this case, the script left to its own devices will leave the MTL contour between the two inlet lines. This can be rectified after processing by deleting the mid-inlet features from the temp file 'shore_2split.' 

In [16]:
# manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
if len(SA_bounds) and not arcpy.Exists(SA_bounds):
    arcpy.CreateFeatureclass_management(home, 'SA_bounds', 'POLYLINE', spatial_reference=utmSR)
print("{} created. \nNow we'll stop for you to manually create lines at each inlet.".format(SA_bounds))

SA_bounds created. 
Now we'll stop for you to manually create lines at each inlet.


In [4]:
bndpoly = fwa.DEMtoFullShorelinePoly(elevGrid, sitevals['MTL'], sitevals['MHW'], inletLines, ShorelinePts)
print('Select features from {} that should not be included in the final shoreline polygon. '.format(bndpoly))

Creating the MTL contour polygon from the DEM...
Aggregation distance: 10
Minimum area: 300
Minimum hole size: 300
Creating the MHW contour polygon from the DEM...
Aggregation distance: 10
Minimum area: 300
Minimum hole size: 300
Combining the two polygons...
...delineating land between MTL and MHW elevations...
...removing the MHW-MTL areas on the oceanside...
User input required! Select extra features in bndpoly for deletion.
Recommended technique: select the polygon/s to keep and then Switch Selection.

Select features from bndpoly that should not be included in the final shoreline polygon. 


*__Requires display in GIS__*

User input is required to identify only the areas within the study area and eliminate isolated landmasses that are not. Once the features to delete are selected, either delete in the GIS or run the code below. Make sure the bndpoly variable matches the layer name in the GIS.

__Do not...__ select the features in ArcGIS and then run DeleteFeatures in this Notebook Python kernel. That will delete the entire feature class. 

```
arcpy.DeleteFeatures_management(bndpoly)
```

The next step snaps the boundary polygon to the shoreline points anywhere they don't already match and as long as as they are within 25 m of each other. 

In [4]:
bndpoly = os.path.join(home, 'bndpoly_manual')  

In [5]:
barrierBoundary = fwa.NewBNDpoly(bndpoly, ShorelinePts, barrierBoundary, '25 METERS', '50 METERS')

Created: bndpoly_manual_2sl ... Should be in your home geodatabase.


#### ShoreBetweenInlets

In [11]:
# This step could be moved out of pre-processing and into the extractor because it doesn't require user input.
shoreline = fwa.CreateShoreBetweenInlets(barrierBoundary, inletLines, shoreline, 
                                         ShorelinePts, proj_code, SA_bounds)

Splitting bndpoly_2sl at inlets...
Preserving only those line segments that intersect shoreline points...
Dissolving the line to create ShoreBetweenInlets...


### Transects - extend, sort, and tidy

Create extendedTrans, which are NASC transects for the study area extended to cover the island, with gaps filled, and sorted in the field sort_ID.

#### 1. Extend the transects and use a copy of the lines to fill alongshore gaps

In [4]:
# Initialize temp file names
trans_extended = os.path.join(arcpy.env.scratchGDB, 'trans_extended')
trans_presort = trans_extended+'_presort'

# Delete transects over 200 m outside of the study area.
if input("Need to remove extra transects? 'y' if barrierBoundary should be used to select. ") == 'y':
    fwa.RemoveTransectsOutsideBounds(orig_trans, barrierBoundary)

# Extend transects and create blank duplicate to use to fill gaps
fwa.ExtendLine(orig_trans, trans_extended, extendlength, proj_code)
fwa.CopyAndWipeFC(trans_extended, trans_presort, ['sort_ID'])
print("MANUALLY: use groups of existing transects in new FC '{}' to fill gaps.".format(trans_presort))

Need to remove extra transects? 'y' if barrierBoundary should be used to select.  n


trans_orig is already projected in UTM.
Transects extended.
MANUALLY: use groups of existing transects in new FC '\\Mac\stor\Projects\DeepDive\TE_vol2\Myrtle\scratch.gdb\trans_extended_presort' to fill gaps.


*__Requires manipulation in GIS__*

1. Edit the trans_presort_temp feature class. __Move and rotate__ groups of transects to fill in gaps that are greater than 50 m alongshore. There is no need to preserve the original transects, but avoid overlapping the transects with each other and with the originals. Do not move any transects slightly. If they are moved, they will not be deleted in the next stage. If you slightly move any, you can either undo or delete that line entirely.

In [5]:
fwa.RemoveDuplicates(trans_presort, trans_extended, barrierBoundary)

Number of features in trans_extended: 51
Number of features in trans_extended_presort: 137
trans_extended_presort ready for sorting. It should be in your scratch geodatabase.


'\\\\Mac\\stor\\Projects\\DeepDive\\TE_vol2\\Myrtle\\scratch.gdb\\trans_extended_presort'

#### 2. Sort the transects along the shore
Usually if the shoreline curves, we need to identify different groups of transects for sorting. This is because the GIS will not correctly establish the alongshore order by simple ordering from the identified sort_corner. If this is the case, answer __yes__ to the next prompt.

In [6]:
sort_lines = fwa.SortTransectPrep(spatialref=utmSR)

Do we need to sort the transects in batches to preserve the order? (y/n)  n
Sort corner (LL, LR, UL, UR):  LL


In [9]:
sort_lines = os.path.join(arcpy.env.scratchGDB, 'sort_lines')

*__Requires manipulation in GIS__*

The last step generated an empty sort lines feature class if you indicated that transects need to be sorted in batches to preserve the order. Now, the user creates lines that will be used to spatially sort transects in groups. 

For each group of transects:

1. __Create a new line__ in 'sort_lines' that intersects all transects in the group. The transects intersected by the line will be sorted independently before being appended to the preceding groups.  (*__add example figure__*)
2. __Assign values__ for the fields 'sort,' 'sort_corner,' and 'reverse.' 'sort' indicates the order in which the line should be used and 'sort_corn' indicates the corner from which to perform the spatial sort ('LL', 'UL', etc.). 'reverse' indicates whether the order should be reversed (roughly equivalent to 'DESCENDING').
3. Run the following code to create a new sorted transect file.

In [10]:
fwa.SortTransectsFromSortLines(trans_presort, extendedTrans, sort_lines, tID_fld)

Creating new feature class extTrans2 to hold sorted transects...
Sorting sort lines by field sort...


TypeError: unsupported operand type(s) for +: 'Result' and 'str'

In [13]:
arcpy.Describe(orig_trans).spatialReference.name
orig_trans = fwa.ReProject(orig_trans, orig_trans+'_utm', proj_code=arcpy.Describe(extendedTrans).spatialReference.factoryCode)
arcpy.Describe(orig_trans).spatialReference.name

The projection of trans_orig was changed. The new file is trans_orig_utm.


'NAD_1983_UTM_Zone_19N'

In [16]:
extendedTrans = os.path.join(home, 'mon_trans')

# project originals to UTM if they're not already
# orig_trans = fwa.ReProject(orig_trans, orig_trans+'_utm', proj_code=arcpy.Describe(extendedTrans).spatialReference.factoryCode)

# If there's no overlap between transect and original transect, set TRANSECTID, TRANSORDER, LRR to fill
fldsToWipe = ['TRANSECTID', 'TRANSORDER', 'LRR']

# 2. Remove orig transects from manually created transects
# If any of the original extended transects (with null values) are still present in trans_presort, delete them.
for row in arcpy.da.SearchCursor(orig_trans, ['SHAPE@']):
    nasc_line = row[0]
    buff = nasc_line.buffer(10)
    with arcpy.da.UpdateCursor(extendedTrans, ['SHAPE@', 'TRANSECTID', 'TRANSORDER']) as cursor:
        for trow in cursor:
            tran = trow[0]
#             if not buff.overlaps(tran): # none of the geometries overlap
#                 trow[1] = -99
#                 trow[2] = -99
# #                 trow[3] = -99
#                 cursor.updateRow(trow)
            if buff.disjoint(tran): # all of them are disjoint even after reprojecting the originals
                trow[1] = -99
                trow[2] = -99
#                 trow[3] = -99
                cursor.updateRow(trow)

## 3. Tidy the extended (and sorted) transects to remove overlap

*__Requires manipulation in GIS__*

Overlapping transects cause problems during conversion to 5-m points and to rasters. We create a separate feature class with the 'tidied' transects, in which the lines don't overlap. This is largely a manually process with the following steps:

1. Select transects to be used to split other transects. Prioritize transects that a) were originally from NASC, b) have dune points within 25 m, and c) are oriented perpendicular to shore. (add example figure)
2. Use the Copy Features geoprocessing tool to copy only the selected transects into a new feature class. If desired, here is the code that could be used to copy the selected features and clear the selection:
```python
arcpy.CopyFeatures_management(extendedTrans, overlapTrans_lines)
arcpy.SelectLayerByAttribute_management(extendedTrans, "CLEAR_SELECTION")
```
3. Run the code below to split the transects at the selected lines of overlap.

In [ ]:
overlapTrans_lines = os.path.join(arcpy.env.scratchGDB, 'overlapTrans_lines')
if not arcpy.Exists(overlapTrans_lines):
    overlapTrans_lines = input("Filename of the feature class of only 'boundary' transects: ")
arcpy.Intersect_analysis([extendedTrans, overlapTrans_lines], trans_x,
                         'ALL', output_type="POINT")
arcpy.SplitLineAtPoint_management(extendedTrans, trans_x, extTrans_tidy)

Delete the extraneous segments manually. Recommended:

1. Using Select with Line draw a line to the appropriate side of the boundary transects. This will select the line segments that need to be deleted.
1. Delete the selected lines.
1. Remove any remaining overlaps entirely by hand. Use the Split Line tool in the Editing toolbar to split lines to be shortened at the points of overlap. Then delete the remnant sections.

## Join anthro data to transects

1. Convert xls spreadsheet to points 
2. Select the first points along each transects and create new FC
3. Spatial Join the new FC to the updated transects 
    - one to one
    - keep all target features
    - keep only the ID fields and the three anthro fields (and the transect fields [LRR, etc.]?)
    - intersect

4. Join the transect values to the pts based on sort_ID


In [ ]:
# Input shapefiles
shlpts_shp = os.path.join(proj_dir, 'rock14_shlpts.shp')
dlpts_shp = os.path.join(proj_dir, 'rock14_dlowpts.shp')
dhpts_shp = os.path.join(proj_dir, 'rock14_dhighpts.shp')
trans_shp = os.path.join(proj_dir, 'rock_trans.shp')
shoreline_shp = os.path.join(proj_dir, 'rock14_shoreline.shp')